<a href="https://colab.research.google.com/github/m37335/kanagawa-exam/blob/master/part3_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install stanza

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# stanza
import stanza
stanza.download('en') # download English model

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma', use_gpu=True, tokenize_pretokenized=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2021-05-05 03:41:52 INFO: Downloading default packages for language: en (English)...
2021-05-05 03:41:53 INFO: File exists: /root/stanza_resources/en/default.zip.
2021-05-05 03:41:57 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-05-05 03:41:57 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-05-05 03:41:57

In [ ]:
def part3_solver(text, candidate):
  possible_answer = []
  tokens = []

  doc = nlp(text)
  for sentence in doc.sentences:
    for word in sentence.words:
      tokens.append(word.text)
  masked_index = tokens.index('*')
  tokens[masked_index] = '[MASK]'
  tokens = ['[CLS]'] + tokens + ['[SEP]']

  print(tokens)

  ids = tokenizer.convert_tokens_to_ids(tokens)
  ids = torch.tensor(ids).reshape(1, -1)

  with torch.no_grad():
    outputs = model(ids)
    # print(outputs)

  predictions = outputs[0]


  _, predicted_indexes = torch.topk(predictions[0][masked_index+1], k=20)
  predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())

  possible_answer.append(predicted_tokens)
  print(possible_answer)
  for i, v in enumerate(predicted_tokens):
    if v in candidate:
      print(i, v)
      break

### H20（ア）
→正解

In [ ]:
text = "New computers are * in our English class."
candidate = ['use', 'uses', 'using', 'used']
# 解答used
part3_solver(text, candidate)

['[CLS]', 'New', 'computers', 'are', '[MASK]', 'in', 'our', 'English', 'class', '.', '[SEP]']
[['not', 'included', 'used', 'available', 'also', 'now', 'located', 'taught', 'only', 'all', 'always', 'here', 'class', 'currently', 'placed', 'found', 'present', 'listed', 'classes', 'still']]
2 used


### H16 （ア）
→正解

In [ ]:
text = "There are a * desks in the room."
candidate = ['few', 'little', 'many', 'lot']
# 解答 few
part3_solver(text, candidate)

['[CLS]', 'There', 'are', 'a', '[MASK]', 'desks', 'in', 'the', 'room', '.', '[SEP]']
[['few', 'number', 'small', 'couple', 'single', '[UNK]', 'little', 'thousand', '.', 'constant', 'hundred', 'letter', 'lot', 'large', 'different', 'character', 'and', 'double', '-', 'big']]
0 few


### H14（イ）
→正解

In [ ]:
text = "Many places in Japan have a * season called tsuyu which comes before summer. It may not be a good season for traveling."
candidate = ['holiday', 'baseball', 'rainy', 'right']
# 解答 rainy
part3_solver(text, candidate)

['[CLS]', 'Many', 'places', 'in', 'Japan', 'have', 'a', '[MASK]', 'season', 'called', 'tsuyu', 'which', 'comes', 'before', 'summer', '.', 'It', 'may', 'not', 'be', 'a', 'good', 'season', 'for', 'traveling', '.', '[SEP]']
[['dry', 'rainy', 'summer', 'long', 'hot', 'winter', 'special', 'good', 'seasonal', 'cold', 'busy', 'short', 'warm', 'cool', 'new', 'wet', 'bad', 'spring', 'different', 'monsoon']]
1 rainy


### H28（エ）
→不正解  
２単語以上の予測はできない。

In [ ]:
text = "The food that you bought yesterday * in a week."
candidate = ['should eat', 'should be eaten', 'has to eat', 'has eaten']
# 解答 should be eaten
part3_solver(text, candidate)

['[CLS]', 'The', 'food', 'that', 'you', 'bought', 'yesterday', '[MASK]', 'in', 'a', 'week', '.', '[SEP]']
[['and', ',', 'morning', 'or', '.', '...', 'once', 'twice', '##s', '-', 'back', 'afternoon', 'is', 'was', 'came', '?', 'day', 'but', 'maybe', 'sometime']]
